<a href="https://colab.research.google.com/github/AuraFrizzati/GoogleCloud-Terraform-EDX-course/blob/main/02_Terraform_Terms_and_Concepts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[02] Terraform Terms & Concepts**

In this module, we introduce you to **HashiCorp Language** and discuss the terms and concepts involved in **authoring a Terraform configuration**.

We also explore some of the important **Terraform commands** involved in managing the terraform configuration.

Upon completion of this module, you will be able to interpret what each code block means, create basic configuration files within Terraform and be able to explain the purpose of a few important Terraform commands and we will also explore what a **Validator tool** is.

Upon completing this module you will be able to

- Explain the **Terraform workflow**.
- Create **basic configuration files** within Terraform.
- Explain the purpose of a few **Terraform commands**.
- Describe the **Terraform Validator tool**.
- **Create, update, and destroy Google Cloud resources** using Terraform.